In [ ]:
# We import all the necessary packages
import os
import re
import collections
import matplotlib.pyplot as plt
import csv
from textblob import TextBlob
import pandas as pd
import sklearn
# import Pickle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve

from sklearn.ensemble import RandomForestClassifier

In [ ]:
path ="/content/drive/MyDrive/"
os.chdir(path)
data = pd.read_csv(f"{path}/emails.csv")

In [ ]:
data.columns


In [ ]:
data['text'].head()

In [ ]:
data['spam'].head()

In [ ]:
data.describe()

Pre-Processing

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:

def tokenize(text):
  words = word_tokenize(text.lower())
  return words

def pre_process(text):
  words = tokenize(text)
  words = [word for word in words if word.isalpha() and word not in stopwords.words('english')]
  return(' '.join(words))




In [ ]:
data['processed_text'] = data['text'].apply(pre_process)
for i in range(3):  # Adjust the range based on the number of examples you want to print
    print("Original:", data['text'][i])
    print("Preprocessed:", data['processed_text'][i])
    print()

In [ ]:
from sklearn.model_selection import train_test_split

messages = data['processed_text']
labels = data["spam"]
data['length'] = data['text'].map(lambda text: len(text))


X_train, X_test, y_train, y_test = train_test_split(messages, labels, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
train_df = pd.DataFrame()
train_df["text"] = X_train
train_df["spam"] = y_train
print("TRAIN DATA")
print(train_df, "\n\n\n")

val_df = pd.DataFrame()
val_df["text"] = X_val
val_df["spam"] = y_val
print("VALIDATION DATA")
print(val_df, "\n\n\n")

test_df = pd.DataFrame()
test_df["text"] = X_test
test_df["spam"] = y_test
print("TEST DATA")
print(test_df)

train_df.to_csv("train.csv", index=False)
val_df.to_csv("validation.csv", index=False)
test_df.to_csv("test.csv", index=False)

In [ ]:
train = pd.read_csv("train.csv")
val = pd.read_csv("validation.csv")
test = pd.read_csv("test.csv")

X_train, y_train = train["text"], train["spam"]
X_val, y_val = val["text"], val["spam"]
X_test, y_test = test["text"], test["spam"]

In [ ]:
data.head()

In [ ]:
bow_transformer = CountVectorizer().fit(train['text'])

In [ ]:
emails_bow = bow_transformer.transform(train['text'])

In [ ]:
tfidf_transformer = TfidfTransformer().fit(emails_bow)

MODEL:1 Naive Bayes Classifier

In [ ]:

# Build and train the model
model = Pipeline([('bow',CountVectorizer()),('tfidf', TfidfTransformer()),('classifier', MultinomialNB())])




In [ ]:
%time model_nb=model.fit(X_train,y_train)

In [ ]:
# Make predictions on the test set
predictions = model_nb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print(classification_report(y_test, predictions))

MODEL 2: Support Vector Machine

In [ ]:

pipeline_sv = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', SVC()),  # train on TF-IDF vectors w/ SVM classifier
])

In [ ]:
%time model_sv = pipeline_sv.fit(X_train, y_train)

In [ ]:

print(model_sv.score(X_train, y_train))
print(model_sv.score(X_val, y_val))

In [ ]:

predictions = model_sv.predict(X_test)
print (confusion_matrix(y_test, predictions))
print (classification_report(y_test, predictions))

MODEL 3: Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier
pipeline_rf = Pipeline([
    ('bow', CountVectorizer()),  # strings to token integer counts
    ('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', RandomForestClassifier()),  # train on TF-IDF vectors w/ SVM classifier
])

In [ ]:
%time model_rf = pipeline_rf.fit(X_train, y_train)

In [ ]:
print(model_rf.score(X_train, y_train))
print(model_rf.score(X_val, y_val))

In [ ]:
predictions = model_rf.predict(X_test)
print (confusion_matrix(y_test, predictions))
print (classification_report(y_test, predictions))

As we can see the SVM gives the best accuracy amongst the three classifiers.